*If time allows (if not,go through this at your own time)*



**Prepatory Step!** For this exercise, we will load and visualise some spatial data. Altair has some useful functions but for this exercise, we will use the geopandas.

First, check if you have geopandas installed by trying to run the following in a cell:

`import geopandas as gpd`

If you don't get  errors, great, skip the installation. 

If you get errors that there is no such library, time to install geopandas.

1 - Stop the Anaconda kernel so that you can install the package.

2 - Go to a terminal and then install geopandas by typing the below into the terminal and pressing enter.

`pip install geopandas`

This should install the package. Now restart Jupyter Notebook and you should be able to run this exercise with no issues.

# Choropleths and how (not) to skew a message with visualisations
A visualisation often shown is a choropleth. This is a series of spatial polygons (such as states in the USA) which are coloured by a feature. Here we will look at creating choropleths of polling data in the recent USA election.

Load in two datasets. One contains the geospatial polygons of the states in America and the other is the polling data we used in the last notebook. 

In [ ]:
import geopandas as gpd 
import pandas as pd
import altair as alt

geo_states = gpd.read_file('gz_2010_us_040_00_500k.json')
df_polls = pd.read_csv('presidential_poll_averages_2020.csv')

Filter our poll data to a specific date.

In [ ]:
df_nov = df_polls[
    (df_polls.modeldate == '11/3/2020')
]

df_nov_states = df_nov[
    (df_nov.candidate_name == 'Donald Trump') |
    (df_nov.candidate_name == 'Joseph R. Biden Jr.')
]

The geo_states variable has polygons for each state.

In [ ]:
geo_states.head()

In [ ]:
alt.Chart(geo_states, title='US states').mark_geoshape().encode(
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

We want to put the percentage estimates for each candidate onto the map. First, let us create a dataframe containing the data for each candidate.

In [ ]:
trump_data = df_nov_states[
    df_nov_states.candidate_name == 'Donald Trump'
]

biden_data = df_nov_states[
    df_nov_states.candidate_name == 'Joseph R. Biden Jr.'
]

Our spatial and poll data have the name of the state in common. We will change the name of the state to NAME to match our geospatial dataframe.

In [ ]:
trump_data.columns = ['cycle', 'NAME', 'modeldate', 'candidate_name', 'pct_estimate', 'pct_trend_adjusted']
biden_data.columns = ['cycle', 'NAME', 'modeldate', 'candidate_name', 'pct_estimate', 'pct_trend_adjusted']

We can join the geospatial and poll data using the NAME column (the name of the state).

In [ ]:
# Create seperate date frame for trump and biden
# Add the poll data
geo_states_trump = geo_states.merge(trump_data, on='NAME')
geo_states_biden = geo_states.merge(biden_data, on='NAME')

In [ ]:
geo_states_trump.head()

In [ ]:
geo_states_biden.head()

Joe Biden is clearly winning. Can we make it look like he is not?

We can plot this specifying the feature to use for our colour.

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    color='pct_estimate',
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

To smooth out any differences we can bin our data.

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate', bin=alt.Bin(step=35)),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

How would you interpret the plot above?

What about if we increase the binstep so we have more bins?

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate', bin=alt.Bin(step=5)),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

Perhaps try different step sizes for the bins and consider how bins can shape our interpretation of the data. What would happen if plots with different bin sizes were placed side to side.

To add further confusion, what happens when we log scale the data?

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate', bin=alt.Bin(step=5), scale=alt.Scale(type='log')),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

vs

In [ ]:
alt.Chart(geo_states_biden, title='Poll estimate for Joe Biden on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate', bin=alt.Bin(step=5), scale=alt.Scale(type='log')),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

What is happening here?!?!

Next up, what about the colours we use and the range of values assigned to each color? Code inspired by/taken from [here](https://colab.research.google.com/drive/1PePamFUfrgvN3ZYaN8fWfP8ovIJ0gyre#scrollTo=Poo1da-8u3cX).

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donal Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate',
    scale=alt.Scale(type="linear",
              domain=[10, 40, 50, 55, 60, 61, 62],
                          range=["#414487","#414487",
                                 "#355f8d","#355f8d",
                                 "#2a788e",
                                 "#fde725","#fde725"])),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

Compare that with

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate',
    scale=alt.Scale(type="linear",
              domain=[10, 20, 30, 35, 68, 70, 100],
                          range=["#414487","#414487",
                                 "#7ad151","#7ad151",
                                 "#bddf26",
                                 "#fde725","#fde725"])),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

My goodness! So what have we played around with?

* Transforming our scale using log
* Binning our data to smooth out variances
* Altering our colour scheme and the ranges for each colour

... what about if we remove the legend?

In [ ]:
alt.Chart(geo_states_trump, title='Poll estimate for Donald Trump on 11/3/2020').mark_geoshape().encode(
    alt.Color('pct_estimate',
    scale=alt.Scale(type="linear",
              domain=[10, 20, 30, 35, 68, 70, 100],
                          range=["#414487","#414487",
                                 "#7ad151","#7ad151",
                                 "#bddf26",
                                 "#fde725","#fde725"]),
                                 legend=None),
    tooltip=['NAME', 'pct_estimate']
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

Good luck trying to interpret that. Though we often see maps without legends and with questionable colour schemes on TV.

How do you think choropleths should be displayed? What information does a use need to understand the message communicated in these plots?